In [1]:
import requests
import json
import pandas as pd
import datetime as dt
from sodapy import Socrata
from scipy.signal import argrelextrema
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

In [2]:
client = Socrata("healthdata.gov", None)

results = client.get("g62h-syeh", limit=50000)

df = pd.DataFrame.from_records(results)

#df = pd.read_csv(r"D:\Henry\PI 2\my_table.csv") #importar desde CSV si no funciona la API

In [3]:
df["date"] = pd.to_datetime(df["date"])
df.sort_values(by="date", inplace=True)
df.reset_index(inplace=True)
df.drop(columns="index",axis=1, inplace=True)

In [4]:
df['Año'] = pd.DatetimeIndex(df['date']).year
df["Mes"] = df.date.dt.strftime('%mm')
df["Año_Mes"] = df["Año"].astype(str)+ "-" +df["Mes"].astype(str)
df=df[(df.date>="2020-01-01") & (df.date<="2022-08-01")]
df.fillna(0, inplace=True)

In [5]:
df = df.astype({"inpatient_beds_used_covid" :int})
df = df.astype({"staffed_adult_icu_bed_occupancy" :int, "staffed_pediatric_icu_bed_occupancy":int})
df["Ocupación_Total_ICU"] = df.staffed_adult_icu_bed_occupancy+df.staffed_pediatric_icu_bed_occupancy
df = df.astype({"staffed_icu_pediatric_patients_confirmed_covid" :int})
df = df.astype({"staffed_icu_adult_patients_confirmed_covid" :int})
df["Total_Confirmados_ICU"] = df.staffed_icu_adult_patients_confirmed_covid+df.staffed_icu_pediatric_patients_confirmed_covid
df = df.astype({"total_pediatric_patients_hospitalized_confirmed_covid" :int})
df = df.astype({"deaths_covid" :int, "critical_staffing_shortage_today_yes": int})

C:\Users\Turko\AppData\Local\Temp\ipykernel_8352\1941413785.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Ocupación_Total_ICU"] = df.staffed_adult_icu_bed_occupancy+df.staffed_pediatric_icu_bed_occupancy
C:\Users\Turko\AppData\Local\Temp\ipykernel_8352\1941413785.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Total_Confirmados_ICU"] = df.staffed_icu_adult_patients_confirmed_covid+df.staffed_icu_pediatric_patients_confirmed_covid


In [6]:
# 1 - ¿Cuáles fueron los 5 Estados con mayor ocupación hospitalaria por COVID? 

df_6m = df[(df.date>="2020-01-01") & (df.date<="2020-06-30")]


Frame_1 = df_6m.groupby("state").sum("inpatient_beds_used_covid").sort_values("inpatient_beds_used_covid", ascending=False)["inpatient_beds_used_covid"].head().to_frame()

#Nota: decido utilizar la columna inpatient_beds_used_covid, a pesar de que incluye pacientes con sola sospecha de covid, porque hay
#estados como Nueva York donde las variables total_adult_patients_hospitalized_confirmed_covid y total_pediatric_patients_hospitalized_confirmed_covid
#comienzan a registrar datos desde julio de 2020, haciendo que resultado sea inapropiado.

fig1_1=px.bar(Frame_1)
fig1_1.update_yaxes(title="", visible=True)
fig1_1.update_xaxes(title="Estado", visible=True)
fig1_1.update_layout(showlegend=False, title="Estados con mayor ocupación hospitalaria por COVID en el primer semestre de 2020")



In [7]:
# 2 - Analice la ocupación de camas (Común) por COVID en el Estado de Nueva York durante la cuarentena establecida e indique:

#Intervalos de crecimiento y decrecimiento
#Puntos críticos (mínimos y máximos)

#Considero el período que va de marzo de 2020 (inicio de la cuarentena) hasta junio de 2021 (levantamiento de las últimas restricciones)

df_cuarentena_NY = df[(df.date>="2020-03-01") & (df.date<="2021-06-30") & (df.state == "NY")]

df_cuarentena_NY.set_index("date", inplace=True)

df_cuarentena_NY_mins = df_cuarentena_NY.iloc[argrelextrema(df_cuarentena_NY.inpatient_beds_used_covid.values, np.less_equal,
                    order=100)[0]]['inpatient_beds_used_covid']
df_cuarentena_NY_maxs = df_cuarentena_NY.iloc[argrelextrema(df_cuarentena_NY.inpatient_beds_used_covid.values, np.greater_equal,
                    order=100)[0]]['inpatient_beds_used_covid']


In [8]:
fig1=px.line(x= df_cuarentena_NY.index, y=df_cuarentena_NY.inpatient_beds_used_covid)
fig2=px.scatter(x=df_cuarentena_NY_mins.index, y=df_cuarentena_NY_mins, color_discrete_sequence=["red"])
fig2.update_traces(marker={'size': 12})
fig3=px.scatter(x=df_cuarentena_NY_maxs.index, y=df_cuarentena_NY_maxs, color_discrete_sequence=["green"])
fig3.update_traces(marker={'size': 12})
fig4=go.Figure(data=fig1.data + fig2.data + fig3.data, layout_title_text='Ocupación de camas por pacientes con COVID en el estado de Nueva York',
layout_title_font_color="black", layout_title_font_size=17)
fig4.show()

In [9]:
print("Los intervalos de crecimiento fueron: \n", "* Desde", df_cuarentena_NY_mins.index[0].date(), "hasta", df_cuarentena_NY_maxs.index[0].date(),
"\n * Desde", df_cuarentena_NY_mins.index[1].date(), "hasta", df_cuarentena_NY_maxs.index[1].date())
print("\n Los intervalos de decrecimiento fueron: \n", "* Desde", df_cuarentena_NY_maxs.index[0].date(), "hasta", df_cuarentena_NY_mins.index[1].date(),
"\n * Desde", df_cuarentena_NY_maxs.index[1].date(), "hasta", df_cuarentena_NY_mins.index[2].date())

Los intervalos de crecimiento fueron: 
 * Desde 2020-03-14 hasta 2020-04-14 
 * Desde 2020-09-20 hasta 2021-01-19

 Los intervalos de decrecimiento fueron: 
 * Desde 2020-04-14 hasta 2020-09-20 
 * Desde 2021-01-19 hasta 2021-06-27


In [10]:
# 3 - ¿Cuáles fueron los cinco Estados que más camas UCI -Unidades de Cuidados Intensivos- 
# utilizaron durante el año 2020? La medición debe realizarse en términos absolutos.

df_2020 = df[(df.date>="2020-01-01") & (df.date<="2020-12-31")]
Frame_3=df_2020.groupby("state").sum("Ocupación_Total_ICU").sort_values("Ocupación_Total_ICU", ascending=False)["Ocupación_Total_ICU"].to_frame().head()

fig3_1=px.bar(Frame_3)
fig3_1.update_yaxes(title="",visible=True)
fig3_1.update_xaxes(title="Estado", visible=True)
fig3_1.update_layout(showlegend=False, title="Estados que más camas UCI -Unidades de Cuidados Intensivos- utilizaron durante el año 2020")

In [11]:
# 4 - ¿Qué cantidad de camas se utilizaron, por Estado, para pacientes pediátricos con COVID durante el 2020?

#total_pediatric_patients_hospitalized_confirmed_covid

Frame_4 = df_2020.groupby("state").sum("total_pediatric_patients_hospitalized_confirmed_covid").sort_values("total_pediatric_patients_hospitalized_confirmed_covid", ascending=False)["total_pediatric_patients_hospitalized_confirmed_covid"].to_frame()

fig4_1=px.bar(Frame_4)
fig4_1.update_yaxes(title="", visible=True)
fig4_1.update_xaxes(title="Estado", visible=True)
fig4_1.update_layout(showlegend=False, title="Cantidad de camas utilizadas, por Estado, para pacientes pediátricos con COVID durante el 2020")


In [12]:
# 5 - ¿Qué porcentaje de camas UCI corresponden a casos confirmados de COVID-19? Agrupe por Estado.

confirmados_porestado = df.groupby("state").sum("Total_Confirmados_ICU").sort_values("state")["Total_Confirmados_ICU"].to_frame()
ICU_porestado = df.groupby("state").sum("Ocupación_Total_ICU").sort_values("state")["Ocupación_Total_ICU"].to_frame()
ICU_porestado = pd.merge(confirmados_porestado, ICU_porestado, how="inner", on="state")
ICU_porestado["%_covid"]=ICU_porestado["Total_Confirmados_ICU"]/ICU_porestado["Ocupación_Total_ICU"]
Frame_5 = ICU_porestado.sort_values("%_covid", ascending=False)["%_covid"].to_frame()

fig5_1=px.bar(Frame_5)
fig5_1.update_yaxes(title="", visible=True)
fig5_1.update_xaxes(title="Estado", visible=True)
fig5_1.update_layout(showlegend=False, title="Porcentaje de camas UCI que corresponden a casos confirmados de COVID-19")

In [13]:
# 6 - ¿Cuántas muertes por covid hubo, por Estado, durante el año 2021?

df_2021 = df[(df.date>="2021-01-01") & (df.date<="2021-12-31")]
Frame_6 = df_2021.groupby("state").sum("deaths_covid").sort_values("deaths_covid", ascending=False)["deaths_covid"].to_frame()

fig6_1=px.bar(Frame_6)
fig6_1.update_yaxes(title="", visible=True)
fig6_1.update_xaxes(title="Estado", visible=True)
fig6_1.update_layout(showlegend=False, title=" Muertes por covid por Estado durante el año 2021")

In [14]:
# 7 - ¿Qué relación presenta la falta de personal médico, con la cantidad de muertes por covid durante el año 2021?

df_2021[["deaths_covid", "critical_staffing_shortage_today_yes"]].corr()

print("presentan una correlación positiva de 57,5%")

presentan una correlación positiva de 57,5%


In [15]:
# 8 - Siguiendo las respuestas anteriores, ¿cuál fue el peor mes de la pandemia para USA en su conjunto? Puede utilizar otras medidas que considere necesarias.

muertes_mensuales=df.groupby("Año_Mes").sum("deaths_covid")
fig8_1= px.line(x=muertes_mensuales.index, y=muertes_mensuales.deaths_covid)
fig8_1.update_yaxes(title="", visible=True)
fig8_1.update_xaxes(title="Estado", visible=True)
fig8_1.update_layout(showlegend=False, title="Muertes de COVID por mes en Estados Unidos")

In [16]:
print("El peor mes fue enero de 2021")

El peor mes fue enero de 2021
